In [1]:
import re
from pathlib import Path

import spacy, pandas as pd

# we're downloading a pre-trained spacy model and loading it
model_loaded: bool = False
while not model_loaded:
    try:
        print("attempting to load model")
        nlp: spacy.Language = spacy.load("en_core_web_sm")
        print("model loaded successfully")
        model_loaded = True
    except BaseException as e:
        print("model failed to load")
        from spacy.cli import download
        download("en_core_web_sm")

attempting to load model
model failed to load
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
attempting to load model
model loaded successfully


In [2]:
path_root: Path = Path(".").resolve()

In [3]:
md_text_raw: str = path_root.joinpath("moby_dick.txt").read_text('utf-8')

# Preprocess the text

In [4]:
# DEFINE SOME USEFUL VARIABLES FOR PREPROCESSING
# utf-8 codes for some characters in the text
dq_op: str = "\u201C" # double quotes open
dq_cl: str = "\u201D" # double quotes close
sq_op: str = "\u2018" # single quote open
sq_cl: str = "\u2019" # single quote close
under: str = "\u005F" # underscore
hyphe: str = "\u002D" # hyphen/minus
mdash: str = "\u2014" # em dash

In [5]:
# replace newlines and carriage returns with whitespace
md_text: str = re.sub("[\n\r]", " ", md_text_raw)
# remove everything before the first chapter
md_text = re.sub("[\w\W]*(?=CHAPTER 1\. Loomings)", "", md_text, count=1)
# remove everything after the end of the epilogue
md_text = re.sub("\s*(\*\*\* END)[\w\W]+", "", md_text, count=1)
# there are some weird embedded books in the middle of the text
# first we remove the in-betweens of the embedded books
md_text = re.sub("(Thus ends BOOK[\w\W]+?)([\w\W]+?BOOK[\w\W]+?)(?=\s\s)", "", md_text)
# then we get rid of the chapter headings of the embedded books
md_text = re.sub("BOOK\s+[IV]+([\w\W]+?(?=CHAPTER)CHAPTER\s+[IV\d]+\.\s+[\w\W]+?(?=\.)\.)?", "", md_text)
# CLEANING UP IMPORTANT UTF-8 CHARACTERS
# replace all utf single quotes with ascii single quotes
md_text = re.sub(f"{sq_op}|{sq_cl}", "\'", md_text)
# replace all utf double quotes with ascii double quotes
md_text = re.sub(f"{dq_op}|{dq_cl}", "\"", md_text)
# replace utf underscores with ascii
md_text = re.sub(f"{under}", "_", md_text)
# replace utf hyphens with ascii
md_text = re.sub(f"{hyphe}", "-", md_text)
# replace utf em dash with hyphen
md_text = re.sub(f"{mdash}", "-", md_text)
# next (and I don't know if this is a good idea...) we're going to replace
# all punctuation that is NOT a:
# . or ! or ? or whitespace or "
md_text = re.sub("[^\w\d\s\.\"?!]", " ", md_text)
# finally, we want to replace repeated whitespace with single whitespace
md_text = re.sub("\s+", " ", md_text)

In [6]:
# defining a function to contain intermediate variables
def split_chapters(text: str) -> dict[str, str]:
    # build a list of the chapter titles and their contents
    list_results: list[str] = [result.strip() for result in re.split("(CHAPTER\s\d+\.\s)|(Epilogue)", text) if result != "" and result is not None]
    # use that list to create a dictionary that is {chapter_title: chapter_content}
    return {x: list_results[i+1] for i, x in enumerate(list_results) if re.match("(CHAPTER\s\d+\.)|(Epilogue)", x)}
# run the processed text through the chapter splitter
dict_chapters: dict[str, str] = split_chapters(md_text)
# dict_chapters

In [7]:
# instantiate the spacy nlp object
# this object contains all of the functionality for
# turning a string or list of strings into spacy "Documents"
"""we no longer need to do the following line as it's loaded with the imports"""
# nlp: spacy.Language = spacy.load("en_core_web_sm")

# our book is a little too long for the default
# processing limit so we increase it slightly
nlp.max_length = 1200000

In [8]:
# let spacy process the full text
# this produces a "Document"
# the document is a tokenized representation of the full
# text with a bunch of extra information attached to the tokens
doc = nlp(md_text)

In [9]:
# spacy stores things like parts of speech as integer tags
# we can use this reverse lookup table to get the string label
lookup: dict[int, str] = {y: x for x, y in spacy.symbols.IDS.items()}

In [10]:
# we can now build a list of all things that are like nouns for the full text
noun_alikes: list[str] = [x.text for x in doc if lookup[x.pos] in {"NOUN", "PROPN", "PRON"}]

In [11]:
# using pandas we build a series from the noun_alikes list
nouns: pd.Series = pd.Series(noun_alikes)
# that we can use to get the unique nouns and how many times they appear in the text
nouns_counts: pd.Series = nouns.value_counts()

In [12]:
nouns_counts

his           2439
it            2193
I             2080
he            1651
him           1055
              ... 
damage           1
northwest        1
nineteenth       1
fur              1
orphan           1
Name: count, Length: 10116, dtype: int64